# Preamble

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg
from matplotlib.patches import Ellipse,Rectangle
import palettable.colorbrewer as pc

from astropy.stats import sigma_clipped_stats
from astropy.io import fits
from photutils import datasets
from photutils import DAOStarFinder
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils import CircularAperture

from cycler import cycler
plt.rcParams["axes.prop_cycle"]=cycler('color',pc.qualitative.Set3_12.mpl_colors)

%matplotlib inline

def plot_moles(FIG,AX,SRCS):
    positions = (SRCS['xcentroid'], SRCS['ycentroid'])
    apertures = CircularAperture(positions, r=50)
    norm = ImageNormalize(stretch=SqrtStretch())
    im=AX.imshow(data, cmap='Greys', origin='lower', norm=norm)
#     ax.set_aspect(0.5)
    apertures.plot(color='C0', lw=3., alpha=0.5)

#     cax = FIG.add_axes([1., 0.14, 0.1, 0.79]) ## [x0,y0,width,height]
#     fig.colorbar(im,cax=cax)
    # print(apertures)

    rect = Rectangle((500,1000),1500,2000,linewidth=5,ls="--",edgecolor='r',facecolor='none') ## xy, width, height
    AX.add_patch(rect)

    AX.set_xticks(range(0,2500,500))
    AX.set_xlabel(r"$x\,/\,{\rm Pixels}$",fontsize=50)
    AX.set_ylabel(r"$y\,/\,{\rm Pixels}$",fontsize=50)

def find_sources(DATA):
    mask_y=[1000,3000]
    mask_x=[500,2000]
    mask = np.ones_like(DATA, dtype=bool)
    # mask[mask_y[0]:mask_y[1],mask_x[0]:mask_x[1]] = False
    mean,median,std=sigma_clipped_stats(DATA,sigma=2.0)    

    daofind = DAOStarFinder(fwhm=10.0, threshold=5.*std)#,sigma_radius=2)#,roundlo=0.1)    
    SOURCES = daofind(DATA-median,mask=mask)    
    for col in SOURCES.colnames:    
        SOURCES[col].info.format = '%.5g'  # for consistent table output
    print(SOURCES)
    return SOURCES

def get_smooth_dist(y,lim="L"):
    if lim=="L":
        MIN,MAX=38.,46.
    else:
        MIN,MAX=3.,251.
    density=ss.gaussian_kde(y)
#     ys = np.linspace(np.min(y)/1.1,np.max(y)*1.1,200)
    ys = np.linspace(MIN,MAX,200)
    density.covariance_factor = lambda : .25
    density._compute_covariance()
    
    dens=density(ys)
    normed=dens/np.max(dens)
    return normed,ys


def plot_key(fig,ax_key,data):
    ax_key.scatter(data,data,label=r"${\bf DATA}$" %locals(),marker=".",facecolor="0.6",edgecolor="none",linewidth=4.,s=500,zorder=-10**8,alpha=1.)
    ax_key.set_xlim([0.,1.])
    ax_key.set_ylim(ax_key.get_xlim())
    leg=ax_key.legend(loc="lower center",framealpha=0.,fontsize=50.,handletextpad=0.1,labelspacing=.5,bbox_to_anchor=(0.35,-0.05))

    cols=["0.6","C0","C2","C3"]
    for i,txt in enumerate(leg.get_texts()):
        txt.set_color(cols[i])


    

## Creating mole dataframe

For code used in region filenames:
FILENAME == regions_NN.reg

- "NN": from "01" -- "14": position used from Mole Screen protocol
 - 15 or 16: added to represent looking down over the left and right shoulders, respectively
 - "FOOT": Foot
 - "FACE": Face
 - if "Z" present, then Zoom in on NN view
 - if "D", image is Dermoscopy
   
Next step is to add to dataframe parameters for images with:
- "R"/"NR": Removed or Not Removed (surgically)
- "S"/"NS": "Scale"/"No Scale": scale/ruler present on image (excluding hairlines in some dermoscopy images)
- "A"/"NA": "Arrow"(s)/"No Arrow"(s): arrow drawn on patient

In [4]:
from regions import read_ds9, write_ds9
from astropy.io import fits
import matplotlib.image as mpimg
import glob
from shutil import copyfile
import pandas as pd

## where to store the fits files
data_path=os.getcwd()+"/0_PATIENT_DATA/"   
    
mole_df=pd.DataFrame(columns=["MOLE_ID",
                              "PATIENT",
                              "PHOTO",
                              "REG_RADIUS_PIXELS",
                              "REG_XCENTER_PIXELS",
                              "REG_YCENTER_PIXELS",
                              "PATIENT_VIEW",
                              "OBS_DATE"])

patient_paths=sorted(glob.glob(data_path+"/*"))
patients=[pat_path.split(data_path)[-1] for pat_path in patient_paths]

for i,patient_path in enumerate(patient_paths):
    patient=patients[i]
    print("PATIENT: "+str(patient))
    
    patient_photo_paths=sorted(glob.glob(patient_path+"/*"))
    patient_photos=[pat_phot_path.split("/"+patient+"/")[-1] for pat_phot_path in patient_photo_paths]
    
    for j,patient_photo_path in enumerate(patient_photo_paths):
        patient_photo=patient_photos[j]
        
        region_paths=glob.glob(patient_photo_path+"/"+patient_photo+"regions/*")
        regions=[reg_path.split("regions/")[-1] for reg_path in region_paths]
        body_positions=[reg.split("_")[-1].split(".reg")[0] for reg in regions]
        
        for k,region_path in enumerate(region_paths):
            region=regions[k]
            body_position=body_positions[k]
            R=read_ds9(region_path)
            
            for l,REGION in enumerate(R):
                MOLE_ID=str(patient)+"_"+str(patient_photo)+"_POS"+str(body_position)+"_"+str(int(l+1))
                temp=pd.DataFrame(data={"MOLE_ID":MOLE_ID,
                                        "PATIENT":patient,
                                        "PHOTO":patient_photo,
                                        "REG_RADIUS_PIXELS":REGION.radius,
                                        "REG_XCENTER_PIXELS":REGION.center.x,
                                        "REG_YCENTER_PIXELS":REGION.center.y,
                                        "PATIENT_VIEW":body_position,
                                        "OBS_DATE":"NaN"},index=[0])
                mole_df=mole_df.append(temp,ignore_index=True)

        
print(mole_df.head())


PATIENT: 0_MASTER_MOLES.csv
PATIENT: A
PATIENT: B
PATIENT: C
PATIENT: D
PATIENT: E
PATIENT: F
PATIENT: G
PATIENT: H
PATIENT: I
PATIENT: J
PATIENT: K
PATIENT: L
PATIENT: M
PATIENT: MOLE_PHOTOMETRY
PATIENT: N
PATIENT: O
PATIENT: P
PATIENT: Q
PATIENT: R
PATIENT: S
           MOLE_ID PATIENT  PHOTO  REG_RADIUS_PIXELS  REG_XCENTER_PIXELS  \
0  A_30536_POS02_1       A  30536          60.735346           1083.1920   
1  A_30536_POS02_2       A  30536          24.238328           1049.7081   
2  A_30536_POS02_3       A  30536          18.615205           1140.1920   
3  A_30536_POS02_4       A  30536          24.238328           1209.2160   
4  A_30536_POS02_5       A  30536          24.238328           2300.9548   

   REG_YCENTER_PIXELS PATIENT_VIEW OBS_DATE  
0           2676.5440           02      NaN  
1           1326.3129           02      NaN  
2           2312.5440           02      NaN  
3           3122.6112           02      NaN  
4           2216.3750           02      NaN  


In [4]:
print((mole_df.loc[mole_df.REG_RADIUS_PIXELS==0.]))
print(len(mole_df))
# mole_df.to_csv("/Users/pboorman/Desktop/mole_df.csv")

Empty DataFrame
Columns: [MOLE_ID, PATIENT, PHOTO, REG_RADIUS_PIXELS, REG_XCENTER_PIXELS, REG_YCENTER_PIXELS, PATIENT_VIEW, OBS_DATE]
Index: []
4009


# Extracting photometry for each mole

Going to link everything FROM the dataframe as a starting point.

This code is quite clunky, but works if there is a clear difference between the mole and skin pixel distributions in r g and b.

In [81]:
## first check that leading zeros have not been removed on the photo filenames which require them!
data_path="/Users/pboorman/Dropbox/1_work/9_projects/9p_molegazer/0_PATIENT_DATA/"   
mole_df=pd.read_csv(data_path+"/0_MASTER_MOLES.csv",converters={'PHOTO': lambda x: str(x)})

## patient E photofilenames all have leading zeros
print(mole_df.loc[mole_df.PATIENT=="E"])

     SCRIPT_ID              MOLE_ID PATIENT     PHOTO  REG_RADIUS_PIXELS  \
513        513   E_00174987_POS01_1       E  00174987          13.217083   
514        514   E_00174987_POS01_2       E  00174987          12.903616   
515        515   E_00174987_POS01_3       E  00174987          17.704148   
516        516   E_00174987_POS01_4       E  00174987          17.785755   
517        517   E_00174987_POS01_5       E  00174987          17.935451   
518        518   E_00174987_POS01_6       E  00174987          10.096859   
519        519   E_00174987_POS01_7       E  00174987          11.372840   
520        520   E_00174987_POS01_8       E  00174987          15.439722   
521        521   E_00174988_POS02_1       E  00174988          21.863241   
522        522   E_00174988_POS02_2       E  00174988          22.730335   
523        523   E_00174988_POS02_3       E  00174988          27.203983   
524        524   E_00174988_POS02_4       E  00174988          28.702468   
525        5

In [21]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from regions import read_ds9,write_ds9,PixCoord,BoundingBox
from astropy.io import fits
from astropy.stats import SigmaClip

import palettable.colorbrewer as pc
from cycler import cycler
plt.rcParams["axes.prop_cycle"]=cycler('color',pc.qualitative.Set3_12.mpl_colors)
%matplotlib inline

from scipy.optimize import curve_fit
from astropy.stats import SigmaClip
from collections import OrderedDict
from astropy.coordinates import Angle, SkyCoord
from regions import PixCoord, CirclePixelRegion



def gauss_x2(x, *p):
    a1, b1, c1, d1, a2, b2, c2, d2=p
    y = a1*np.exp(-np.power((x - b1), 2.)/(2. * c1**2.)) +a2*np.exp(-np.power((x - b2), 2.)/(2. * c2**2.))# + d1+ d2
    return y

def gauss_x1(x, *p):
    a1, b1, c1, d1=p
    y = a1*np.exp(-np.power((x - b1), 2.)/(2. * c1**2.))# + d1
    return y


def FIT_GAUSSIAN(X,Y):#,AX,PLOT_LEG):
#     fig,ax1=plt.subplots(figsize=(15,12))
    #     [amplitude, mean, sigma, vertical normalisation]
    PEAK_initial = [np.median(Y), int((np.max(X)-np.min(X))/3.+np.min(X)), 4., 0.,
                    np.median(Y), X[np.argmax(Y)], 4., 0.]
    try:
        popt,pcov=curve_fit(gauss_x2, X, Y, p0=PEAK_initial,maxfev=1000)#, bounds=param_bounds)#, sigma=e)

    except (RuntimeError,TypeError):
        return np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
        
#     y_initial=gauss_x1(X, *PEAK_initial[0:4])
#     ax1.plot(X,y_initial,'C6',ls="--",marker=",",linewidth=4.,label=r"${\rm Input\ Mole}$",alpha=0.3)
#     y_initial=gauss_x1(X, *PEAK_initial[4:])
#     ax1.plot(X,y_initial,'C6',ls=":",marker=",",linewidth=4.,label=r"${\rm Input\ Skin}$",alpha=0.3)
    
    y_fit_MOLE = gauss_x1(X, *popt[0:4])
#     AX.plot(X,y_fit_MOLE,'k',ls="--",marker=",",linewidth=4.,label=r"${\rm Fit\ Mole}$")
    MOLE_MEAN=popt[1]
    MOLE_SIGMA=popt[2]
    MOLE_AMPLITUDE=popt[0]
    SKIN_MEAN=popt[5]
    SKIN_SIGMA=popt[6]
    SKIN_AMPLITUDE=popt[4]
    
    
    y_fit_SKIN = gauss_x1(X, *popt[4:])
#     AX.plot(X,y_fit_SKIN,'k',ls=":",marker=",",linewidth=4.,label=r"${\rm Fit\ Skin}$")
    
    ## by using the zeroth INTERSECT, we are assuming that the mole is darker and
    ## hence to the left of the skin distribution
    try:
        INTERSECT=np.argwhere(np.diff(np.sign(y_fit_MOLE-y_fit_SKIN))).flatten()[0]
#         AX.plot(X[INTERSECT],y_fit_MOLE[INTERSECT],"ro")
#         AX.fill_between(X[X>X[INTERSECT]],0.,y_fit_SKIN[X>X[INTERSECT]],facecolor="k")
    except:
        INTERSECT=np.nan
#         print("No intersections found -- mole might be quite faint...")
    
#     ax1.plot(X,Y,'C9',marker="o",linewidth=4.,label=r"${\rm Data}$",alpha=0.5)

#     if PLOT_LEG:
#         leg=AX.legend(ncol=2,bbox_to_anchor=(.9, 1.3))
    return INTERSECT,MOLE_MEAN,MOLE_SIGMA,MOLE_AMPLITUDE,SKIN_MEAN,SKIN_SIGMA,SKIN_AMPLITUDE




data_path="/Users/pboorman/Dropbox/1_work/9_projects/9p_molegazer/0_PATIENT_DATA/"   
mole_df=pd.read_csv(data_path+"/0_MASTER_MOLES.csv",converters={'PHOTO': lambda x: str(x)})

mole_df[["red_SKIN",
         "dred_SKIN",
         "AMP_red_SKIN",
         "green_SKIN",
         "dgreen_SKIN",
         "AMP_green_SKIN",
         "blue_SKIN",
         "dblue_SKIN",
         "AMP_blue_SKIN"]]=pd.DataFrame([np.repeat(np.nan,9)], index=mole_df.index)

mole_df[["red_MOLE",
         "dred_MOLE",
         "AMP_red_MOLE",
         "green_MOLE",
         "dgreen_MOLE",
         "AMP_green_MOLE",
         "blue_MOLE",
         "dblue_MOLE",
         "AMP_blue_MOLE"]]=pd.DataFrame([np.repeat(np.nan,9)], index=mole_df.index)



def EXTRACT_MOLE(fits_file):
    IMAGE=fits.open(fits_file)[0].data
    MASK=REGION.to_mask(mode="exact")
    MOLE_DATA=-MASK.cutout(-IMAGE)

    NPIX=int(np.size(MOLE_DATA)/12)
    N,BINS=np.histogram(MOLE_DATA,bins=NPIX)
    BC=(BINS[0:-1]+BINS[1:])/2.
    BC=BC[N>0.];N=N[N>0.]
    return BC,N
        


## this parameter is used to store the region radius required to fit two Gaussians to the mole
mole_df.loc[:,"FIT_REG_RADIUS_PIXELS"]=mole_df.REG_RADIUS_PIXELS
# mole_df=mole_df.tail(1400)
for i,src in mole_df.loc[mole_df.PATIENT_VIEW!="D"].iterrows():
    if i % 5 == 0:print(i)
    PHOTO_PATH=data_path+"/"+str(src.PATIENT)+"/"+str(src.PHOTO)+"/"+str(src.PHOTO)
    for j,BAND in enumerate(["red","green","blue"]):
        
        
        center=PixCoord(x=src.REG_XCENTER_PIXELS,y=src.REG_YCENTER_PIXELS)
        radius=src.REG_RADIUS_PIXELS
        INTER="nan"
        count=0
        while str(INTER)=="nan":
            if count>20:
                print("    Solution not found! May just be a faint mole...")
                break
            radius*=(1.+float(count)*0.05)
            if count>0:print("   Trying with radius "+str(int(100.*float(count)*0.05))+" per cent larger")
                
            REGION=CirclePixelRegion(center,radius)

            BC,N=EXTRACT_MOLE(PHOTO_PATH+"%(BAND)s.fits" %locals())
            INTER,MEAN_MOLE,SIGMA_MOLE,AMPLITUDE_MOLE,MEAN_SKIN,SIGMA_SKIN,AMPLITUDE_SKIN=FIT_GAUSSIAN(BC,N)
    
            if str(INTER)!="nan" and count>0:
                print("    Success! Re-saving region radius...")
                mole_df.loc[mole_df.SCRIPT_ID==i,"FIT_REG_RADIUS_PIXELS"]=radius
            count+=1
                
#         print(INTER)        
        mole_df.loc[mole_df.SCRIPT_ID==i,BAND+"_SKIN"]=MEAN_SKIN
        mole_df.loc[mole_df.SCRIPT_ID==i,"d"+BAND+"_SKIN"]=SIGMA_SKIN
        mole_df.loc[mole_df.SCRIPT_ID==i,"AMP_"+BAND+"_SKIN"]=AMPLITUDE_SKIN
        
        mole_df.loc[mole_df.SCRIPT_ID==i,BAND+"_MOLE"]=MEAN_MOLE
        mole_df.loc[mole_df.SCRIPT_ID==i,"d"+BAND+"_MOLE"]=SIGMA_MOLE
        mole_df.loc[mole_df.SCRIPT_ID==i,"AMP_"+BAND+"_MOLE"]=AMPLITUDE_MOLE
    if count==0:print()
    
print(mole_df.head()) 
    
       

0
   Trying with radius 5 per cent larger
    Success! Re-saving region radius...
   Trying with radius 5 per cent larger
    Success! Re-saving region radius...
   Trying with radius 5 per cent larger
   Trying with radius 10 per cent larger
   Trying with radius 15 per cent larger
   Trying with radius 20 per cent larger
   Trying with radius 25 per cent larger
    Success! Re-saving region radius...
5
   Trying with radius 5 per cent larger
   Trying with radius 10 per cent larger
   Trying with radius 15 per cent larger
   Trying with radius 20 per cent larger
    Success! Re-saving region radius...
   Trying with radius 5 per cent larger
   Trying with radius 10 per cent larger
    Success! Re-saving region radius...
   Trying with radius 5 per cent larger
    Success! Re-saving region radius...
   Trying with radius 5 per cent larger
    Success! Re-saving region radius...
   Trying with radius 5 per cent larger
    Success! Re-saving region radius...
10
20
25
   Trying with radi

In [22]:
mole_df.to_csv("/Users/pboorman/Dropbox/moles_with_photometry.csv")

## Now plot figure for each mole in R, G and B (on same plot)

Try on one mole:

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from regions import read_ds9,write_ds9,PixCoord,BoundingBox,CirclePixelRegion
from astropy.io import fits
from astropy.stats import SigmaClip
import palettable.colorbrewer as pc
from cycler import cycler
from scipy.optimize import curve_fit
from astropy.stats import SigmaClip
from collections import OrderedDict

plt.rcParams["axes.prop_cycle"]=cycler('color',pc.qualitative.Set3_12.mpl_colors)
%matplotlib inline

data_path=os.getcwd()+"/0_PATIENT_DATA/"   
mole_df=pd.read_csv(data_path+"/0_MASTER_MOLES.csv",converters={'PHOTO': lambda x: str(x)})


## filter moles that photometry could not be extracted for
legit_moles=mole_df.dropna(subset=["red_SKIN",
                                   "dred_SKIN",
                                   "AMP_red_SKIN",
                                   "green_SKIN",
                                   "dgreen_SKIN",
                                   "AMP_green_SKIN",
                                   "blue_SKIN",
                                   "dblue_SKIN",
                                   "AMP_blue_SKIN",
                                   "red_MOLE",
                                   "dred_MOLE",
                                   "AMP_red_MOLE",
                                   "green_MOLE",
                                   "dgreen_MOLE",
                                   "AMP_green_MOLE",
                                   "blue_MOLE",
                                   "dblue_MOLE",
                                   "AMP_blue_MOLE"])

print(len(legit_moles))


3920


Insert leading zeros by hand

(Warning: takes a while!)

In [6]:
patient="F"
photo="29010"
mole="10"


## BE CAREFUL -- currently just inserting leading zeros by hand to photo names with > 5 characters
def LEADING_ZEROS(row):
    if len(row["PHOTO"]) > 5:
        return (row["PHOTO"]).zfill(8)
    else:
        return row["PHOTO"]


legit_moles.loc[:,"PHOTO"]=legit_moles.apply(LEADING_ZEROS, axis=1)
legit_moles.loc[:,"PHOTO_PATH"]=data_path+"/"+legit_moles.PATIENT+"/"+legit_moles.PHOTO+"/"+legit_moles.PHOTO

mole_ID="I_39130_POS15_9"
MOLE=legit_moles.loc[(legit_moles.MOLE_ID==mole_ID)]
print(MOLE.SCRIPT_ID)

## use this to start the figure generation on a particular mole (i.e. miss out the ones that figures have already been generated for)
legit_moles=legit_moles.loc[(legit_moles.SCRIPT_ID>1261)]
print(len(legit_moles))

1262    1262
Name: SCRIPT_ID, dtype: int64
2701


In [ ]:
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import matplotlib.image as mpimg
import matplotlib.patches as patches
import numpy as np



def gauss_x1(x, *p):
    a1, b1, c1, d1=p
    y = a1*np.exp(-np.power((x - b1), 2.)/(2. * c1**2.))# + d1
    return y


def FIND_INSET_LOC(xc,yc,xhalf,yhalf):
    if xc < xhalf and yc < yhalf:
        location="upper right"
    elif xc < xhalf and yc > yhalf:
        location="lower right"
    elif xc > xhalf and yc < yhalf:
        location="upper left"
    else:
        location="lower left"
    
    return location

def ZOOM_IN_RECT(input,AX,ZOOM,rect,cmap):
    x1, x2, y1, y2 = rect.get_x(), rect.get_x()+rect.get_width(),rect.get_y(), rect.get_y()+rect.get_height() # specify the limits
    
    LOC=FIND_INSET_LOC((x1+x2)/2.,(y1+y2)/2.,np.shape(input)[1]/2.,np.shape(input)[0]/2.)
    axins=zoomed_inset_axes(AX,ZOOM,loc=LOC)#bbox_to_anchor=(0.7,0.9),bbox_transform=AX.transAxes, loc=3, borderpad=0)
    
    axins.set_xlim(x1, x2)  # apply the x-limits
    axins.set_ylim(y1, y2)  # apply the y-limits
    mark_inset(AX, axins, loc1=3, loc2=4, fc="none", ec="1.")
    plt.yticks(visible=False)
    plt.xticks(visible=False)
    # flip image
    rot = np.flipud(input)
    axins.imshow(input, cmap=cmap,zorder=10**6)
    for spine in axins.spines.values():
        spine.set_edgecolor('white')
        spine.set_linewidth(4.)



for i,MOLE in legit_moles.iterrows():
    fig=plt.figure(figsize=(28,12))
    gs=gridspec.GridSpec(3,7)
    gs.update(left=0.,right=1.,bottom=0.,top=0.98,wspace=0.07,hspace=0.025)
    ax_huma=fig.add_subplot(gs[0:3,0:3])

    for j,band in enumerate(["red","green","blue"]):
        ax_data=fig.add_subplot(gs[j,3])
        ax_hist=fig.add_subplot(gs[j,4:6])
        ax_filt=fig.add_subplot(gs[j,6])

        PRELIM_DATA=fits.open(str(MOLE.PHOTO_PATH)+band+".fits")[0].data
        REG=CirclePixelRegion(PixCoord(x=float(MOLE.REG_XCENTER_PIXELS), y=float(MOLE.REG_YCENTER_PIXELS)), radius=float(MOLE.FIT_REG_RADIUS_PIXELS))
        MASK=REG.to_mask(mode="exact")
        MOLE_DATA=MASK.cutout(PRELIM_DATA)
        NPIX=int(np.size(MOLE_DATA)/12)
        N,BINS=np.histogram(MOLE_DATA,bins=NPIX)
        CENTER_PIXEL=(MOLE_DATA[int(len(MOLE_DATA)/2),int(len(MOLE_DATA)/2)])
        BC=(BINS[0:-1]+BINS[1:])/2.
        BC=BC[N>0.];N=N[N>0.]

        y_fit_MOLE=gauss_x1(BC,MOLE["AMP_"+band+"_MOLE"],MOLE[band+"_MOLE"],MOLE["d"+band+"_MOLE"],0.)
        y_fit_SKIN=gauss_x1(BC,MOLE["AMP_"+band+"_SKIN"],MOLE[band+"_SKIN"],MOLE["d"+band+"_SKIN"],0.)

        FILTERED=MOLE_DATA
        

        image=mpimg.imread(str(MOLE.PHOTO_PATH)+"ORIGINAL.jpg")[::-1]
        ax_huma.imshow(image)
        ax_huma.set_ylim(ax_huma.get_ylim()[::-1])
        artist=REG.as_artist()
#         ax_huma.add_artist(artist)
        lower_corner=(float(MOLE.REG_XCENTER_PIXELS)-float(MOLE.REG_RADIUS_PIXELS),float(MOLE.REG_YCENTER_PIXELS)-float(MOLE.REG_RADIUS_PIXELS))
        rect1 = patches.Rectangle(lower_corner,float(MOLE.REG_RADIUS_PIXELS)*2.,float(MOLE.REG_RADIUS_PIXELS)*2.,linewidth=3, edgecolor='r', facecolor='none')
        ZOOM_FACTOR=500./float(MOLE.REG_RADIUS_PIXELS)
        ZOOM_IN_RECT(image,ax_huma,ZOOM_FACTOR,rect1,cmap='gray')
        ax_huma.set_xlabel(r"$x$")
        ax_huma.set_ylabel(r"$y$")



        ax_data.imshow(-MOLE_DATA,interpolation='nearest', origin='lower',extent=MASK.bbox.extent,cmap="Greys")
        ax_data.set_xlabel(r"$x$")
        ax_data.set_ylabel(r"$y$")
        for tick in ax_data.get_xticklabels():
            tick.set_rotation(45)
        for tick in ax_data.get_yticklabels():
            tick.set_rotation(45)

        ax_hist.plot(BC,y_fit_MOLE,'k',ls="--",marker=",",linewidth=4.,label=r"${\rm Fit\ Mole}$")
        ax_hist.plot(BC,y_fit_SKIN,'k',ls=":",marker=",",linewidth=4.,label=r"${\rm Fit\ Skin}$")
        ax_hist.plot(BC,N,ls='steps-mid',c=band,alpha=0.5)
        
        try:
            INTER=np.argwhere(np.diff(np.sign(y_fit_MOLE-y_fit_SKIN))).flatten()[0]
            ax_hist.fill_between(BC[BC>BC[INTER]],0.,N[BC>BC[INTER]],facecolor="0.7",zorder=-1)
            FILTERED[MOLE_DATA>BC[INTER]]=0.
            ax_filt.imshow(-FILTERED,
                           interpolation='nearest', origin='lower',
                           extent=MASK.bbox.extent,cmap="Greys",vmin=-BC[INTER],vmax=-np.min(BC))
        except IndexError:
            print("No intersect found?")
        ax_hist.set_xlim(0.,255.)#np.min(TIME),np.max(TIME))
        ax_hist.set_ylim(0.,np.max(N)*1.02)
        ax_hist.set_yticklabels([])
        ax_hist.set_xlabel(r"${\rm Brightness}$")
        for tick in ax_hist.get_xticklabels():
            tick.set_rotation(45)

        ax_filt.yaxis.tick_right()
        ax_filt.yaxis.set_label_position("right")
        ax_filt.yaxis.set_ticks_position('both')

        ax_filt.set_xlabel(r"$x$")
        ax_filt.set_ylabel(r"$y$")
        for tick in ax_filt.get_xticklabels():
            tick.set_rotation(45)
        for tick in ax_filt.get_yticklabels():
            tick.set_rotation(45)

        if int(j)<2:
            ax_data.set_xticklabels([])
            ax_hist.set_xticklabels([])
            ax_filt.set_xticklabels([])

    TITLE="Patient: "+str(MOLE.PATIENT)+", Photo: "+str(MOLE.PHOTO)+", Mole: "+str(int(MOLE.SCRIPT_ID)+1)+"/"+str(len(legit_moles))+", Date: "+str(MOLE.OBS_DATE)
    fig.text(0.5,1.03,TITLE,ha='center',va='bottom',fontsize=60.,color='k')
    MOLE_NAME=MOLE.MOLE_ID
    print(MOLE_NAME)
    fig.savefig(data_path+"/MOLE_PHOTOMETRY/"+MOLE_NAME+".png")
    plt.close()


F_79946_POS11_8
F_79946_POS11_9
F_79947_POS09_1
F_79947_POS09_2
F_79947_POS09_3
F_79947_POS09_4
F_79947_POS09_5
F_79947_POS09_6
F_79947_POS09_7
F_79948_POS12_1
F_79948_POS12_2
F_79948_POS12_3
F_79948_POS12_4
F_79948_POS12_5
F_79949_POS10_1
F_79949_POS10_2
F_79949_POS10_3
F_79949_POS10_4
F_79949_POS10_5
F_79949_POS10_6
F_79949_POS10_7
F_79949_POS10_8
F_79949_POS10_9
F_79950_POS05_1
F_79950_POS05_2
F_79950_POS05_3
F_79950_POS05_4
F_79950_POS05_5
F_79950_POS05_6
F_79950_POS05_7
F_79950_POS05_8
F_79950_POS05_9
F_79951_POS06_1
F_79951_POS06_2
F_79951_POS06_3
F_79951_POS06_4
F_79951_POS06_5
F_79951_POS06_6
F_79951_POS06_7
F_79951_POS06_8
F_79951_POS06_9
F_79951_POS06_10
F_79952_POS07_1
F_79952_POS07_2
F_79952_POS07_3
F_79952_POS07_4
F_79952_POS07_5
F_79952_POS07_6
F_79953_POS08_1
F_79953_POS08_2
F_79953_POS08_3
F_79953_POS08_4
F_79953_POS08_5
G_28834_POS01_1
G_28834_POS01_2
G_28834_POS01_3
G_28834_POS01_4
G_28834_POS01_5
G_28834_POS01_6
G_28834_POS01_7
G_28834_POS01_8
G_28834_POS01_9
G_28834